In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
import pandas as pd

def read_conll(filename):
    df = pd.read_csv(filename,
                     sep=',', header=None, keep_default_na=False,
                     names=['sentence_id', 'words', 'pos', "labels"],
                     quoting=1, skip_blank_lines=False, index_col=False)
    return df


def read_conll_val(filename):
    df = pd.read_csv(filename,
                     sep=',', header=None, keep_default_na=False,
                     names=['sentence_id', 'words', "labels"],
                     quoting=1, skip_blank_lines=False, index_col=False)
    return df

In [ ]:
train_df = read_conll('content/data/train.csv')
test_df = read_conll_val('content/data/val.csv')
dev_df = read_conll_val('content/data/val.csv')
test_df

In [ ]:
custom_labels = list(train_df['labels'].unique())
custom_labels

In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 128,
    'num_train_epochs': 10,
    'train_batch_size': 32,
    'no_save': True,
    'fp16': True,
    'output_dir': '/outputs/',
    'best_model_dir': '/outputs/best_model/',
    'evaluate_during_training': True,
}

In [ ]:
train_args

In [ ]:
from simpletransformers.ner import NERModel
import pandas as pd
import logging

logging.basicConfig(level=logging.DEBUG)
# transformers_logger = logging.getLogger('transformers')
# transformers_logger.setLevel(logging.WARNING)

# We use the bio BERT pre-trained model.
model = NERModel('bert', 'dmis-lab/biobert-v1.1', labels=custom_labels, args=train_args, use_cuda=False)

# Train the model
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df, eval_data=dev_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, preds_list = model.eval_model(test_df)

In [ ]:
sample = "Hello ai , when I wake up in this morning . I had a slight vertigo . After I ate breakfast , I have some shortness of breath and fall ."

samples = [sample]
predictions, _ = model.predict(samples)
for predict in predictions[0]:
    print(predict)

In [ ]:
from simpletransformers.ner import NERModel

model_best = NERModel('bert', 'outputs/best_model', use_cuda=False)

result, model_outputs, preds_list = model_best.eval_model(test_df)
result